<a href="https://colab.research.google.com/github/fladdict/stable-diffusion/blob/main/super_simple_stable_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SSSD, Super Simple Stable Diffusion

Lightwaight easy to use Stable Diffusion Kit.
Just run all cells. 

Only you need is [Hugging Face's user setting page](https://huggingface.co/settings/tokens).

## LICENSE
[Stable Diffusion](https://github.com/CompVis/stable-diffusion/blob/main/LICENSE)

[Diffuser](https://github.com/huggingface/diffusers/blob/main/LICENSE)

## Setup Google Drive (If needed)

## Install Library

In [ ]:
#Install Lib
!pip install diffusers==0.4.0 transformers scipy ftfy

## Login to Hugging Face

First you need go to Huggingface and accept [stable diffusion model's LICENCE ](https://huggingface.co/spaces/CompVis/stable-diffusion-license)

Get ACCESS TOKEN from [Huggingface's user setting page](https://huggingface.co/settings/tokens).

Run a cell below and fill ACCESS TOKEN (read) into the login form.

In [2]:
#Get access token from Hugging Face
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


## Function Setup

In [3]:
import torch
from diffusers import StableDiffusionPipeline
from diffusers import StableDiffusionImg2ImgPipeline
import numpy as np
import PIL
import gc
import os

def save_image(directory_path, file_name, img):
  os.makedirs(directory_path, exist_ok = True)
  img.save(f"{directory_path}/{file_name}")



def clear_memory():
  gc.collect()
  torch.cuda.empty_cache()



def get_txt2img_pipeline(model_path = "CompVis/stable-diffusion-v1-4"):
  pipeline = StableDiffusionPipeline.from_pretrained(model_path, 
                                                     revision="fp16", 
                                                     torch_dtype=torch.float16,
                                                     use_auth_token=True)
  pipeline.to("cuda")
  pipeline.enable_attention_slicing()
  return pipeline




def get_img2img_pipeline(model_path = "CompVis/stable-diffusion-v1-4"):
  pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(model_path, 
                                                            revision="fp16", 
                                                            torch_dtype=torch.float16,
                                                            use_auth_token=True)
  pipeline.to("cuda")
  pipeline.enable_attention_slicing()
  return pipeline

pipe = None
last_model_path = None

In [4]:
#Additional Feature
# if you use UPPER CASE text in a prompt, it randomly swap the woard from this dictionary.

decorate_dict = {
    "COLOR":["red", "green", "blue", "yellow", "orange", "purple", "black", "white", "dark", "blight", "gold", "silver", "brown"],
    "GENDER":["male","female","unisex"],
    "MODIFIER":["UnrealEngine","Octane Render", "8k", "High Qulity", "Beautiful", "Art Lens", "Award Wining", "Pixiv Trending", "Masterpiece", "Epic"]
}


def decorate_prompt(prompt, dict):
    for index, key in enumerate(dict):
      while(prompt.find(key)!=-1):
        words = dict[key];
        word = words[random.randint(0,len(words)-1)];
        prompt = prompt.replace(key, word, 1);
    return prompt;

# Diffuse!!!

Once you setup, just run only this cell

In [ ]:
from torch import autocast
from datetime import datetime
import random
from PIL import Image

#@markdown ## General Settings
prompt = "" #@param {type:"string"}
negative_prompt = "blur, unsharp, unclear, noise, rough sketch, speed painting, bad drawing" #@param {type:"string"}
width = 768 #@param {type:"integer"}
height = 512 #@param {type:"integer"}

guidance_scale = 7.5 #@param {type:"number"}
num_inference_steps = 50 #@param {type:"number"}
seed = None #@param
num = 3 #@param {type:"integer"}

#@markdown -
#@markdown ## Init Image settings
#@markdown File path used for base Image. 
init_image = "/content/download-9.png" #@param {type:"string"}
strength = 0.9 #@param {type:"number"}

#@markdown - 
#@markdown ## Loopback
#@markdown If True, it takes time but high quality
is_loopback = True #@param {type:"boolean"}
loopback_num = 2 #@param {type:"integer"}

#@markdown -
#@markdown ## Model Settings
#@markdown Select huggingface's model card model path

model_path = "CompVis/stable-diffusion-v1-4" #@param {type:"string"} ["CompVis/stable-diffusion-v1-4", "naclbit/trinart_stable_diffusion_v2", "hakurei/waifu-diffusion"] {allow-input: true}

#@markdown -
#@markdown ## Export Settings

save_directory_path = "/content/output/test" #@param {type:"string"}
save_file_prefix = "SD" #@param {type:"string"}
is_save_image = True #@param {type:"boolean"}
is_display_image = True #@param {type:"boolean"}

clear_memory()

with autocast("cuda"):
  if init_image != None:
    _init_image = Image.open(init_image)
  for index in range(num):
    _seed = random.randint(0, 2**32) if seed == None else seed
    generator = torch.Generator("cuda").manual_seed(_seed)

    _prompt = decorate_prompt(prompt, decorate_dict)
    _negative_prompt = decorate_prompt(negative_prompt, decorate_dict)

    if init_image != None and init_image != "" :
      if pipe == None or type(pipe) != StableDiffusionImg2ImgPipeline or model_path != last_model_path:
        pipe = get_img2img_pipeline(model_path)
        last_model_path = model_path
        
      result = pipe(
          init_image = _init_image,
          prompt = _prompt,
          negative_prompt = _negative_prompt,
          width = width,
          height = height,
          strength = strength,
          guidance_scale = guidance_scale,
          num_inference_steps = num_inference_steps,
          generator = generator
      )
    else:
      if pipe == None or type(pipe) != StableDiffusionPipeline or model_path != last_model_path:
        pipe = get_txt2img_pipeline(model_path)
        last_model_path = model_path
      result = pipe(
          prompt = _prompt,
          negative_prompt = _negative_prompt,
          width = width,
          height = height,
          strength = strength,
          guidance_scale = guidance_scale,
          num_inference_steps = num_inference_steps,
          generator = generator
      )

    image = result["sample"][0]
    is_nswf = result["nsfw_content_detected"][0]

    if is_display_image == True:
      display(image)

    if is_save_image == True and is_nswf==False:
      file_id = datetime.today().strftime('%Y-%m-%d-%H-%M-%S')
      save_image(save_directory_path, f"{save_file_prefix}-{file_id}.png", image)



# Cheat Sheet



## Prompt for beginners


```
<MEDIUM> of <THEME> by <ARTIST> in the style of <STYLE>, <MODIFIER>, <MODIFIER>
```

```
Oil painting of landscape by Rembrandt in the style of impressionism, high quality, masterpiece
```

## Style
* Ancient Egyptian Art
* Ancient Greek Art
* Byzantine Art
* Gothic
* Medieval Art
* Renaissance
* Mannerism
* High Renaissance
* Baroque
* Rococo
* Classicism
* Neo Classicism
* Academicism
* Romanticism
* Orientalism
* Realism
* Primitivism
* Surrealism
* Impressionism
* Symbolism
* Pointillism
* Abstract Art
* Japonism
* Art Deco
* Fauvism
* Art Nouveau
* Expressionism
* Cubism
* Futurism
* Dada
* Constructivism
* Outsider art
* Concretism
* Neo Expressionism
* Pop Art
* Modernism
* Minimalism
* Photorealism
* Hyper-Realism

## Artist
* Rembrandt


## Modifier
* Masterpiece of
* High quality
* Photorealistic
* Unrealengine
* Octane Render
* 8k
* Insane detailing
* Epic